In [6]:
import pandas as pd
import importlib
import matplotlib.pyplot as plt
import stumpy
import numpy as np
import sys
sys.path.append('..')
from src.utils import utils
importlib.reload(utils)
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

<module 'src.utils.utils' from 'c:\\LocalData\\pabflore\\encoder-pacman\\notebooks\\..\\src\\utils\\utils.py'>

In [ ]:
user_df, ip_df, redcap_df, game_df, gamestate_df, psychometrics_df = utils.read_data('../data/')

gamestate_df.columns

FileNotFoundError: [Errno 2] No such file or directory: 'data/userip.csv'

In [18]:
GAME_ID = 716

GAME_STATE_VARIABLES = ['score', 'powerPellets']
COORDINATES = ['Pacman_X', 'Pacman_Y']
DIRECTION = ['movement_direction']
INPUT = ['input_direction']


pacman_ts = gamestate_df.loc[gamestate_df['game_id'] == GAME_ID][['time_elapsed'] + COORDINATES + DIRECTION]
pacman_ts.reset_index(drop=True, inplace=True)

for col in COORDINATES:
    pacman_ts[col] = pacman_ts[col].astype(float)

# pacman_ts = utils.pos_mirroring(pacman_ts, return_quadrant=True)

pacman_ts.head()

,time_elapsed,Pacman_X,Pacman_Y,movement_direction
0,0.20,0.0,-9.5,none
1,0.25,0.0,-9.5,none
2,0.30,0.0,-9.5,none
3,0.35,0.0,-9.5,none
4,0.39,0.0,-9.5,none


In [19]:
pacman_ts['position_vector'] = list(zip(pacman_ts['Pacman_X'], pacman_ts['Pacman_Y']))

direction_mapping = {
    'right': (1, 0),
    'left': (-1, 0),
    'up': (0, 1),
    'down': (0, -1),
    'none': (0, 0)
}

pacman_ts['movement_vector'] = pacman_ts['movement_direction'].map(direction_mapping)


pacman_ts

,time_elapsed,Pacman_X,Pacman_Y,movement_direction,position_vector,movement_vector
0,0.200000,0.00,-9.50,none,"(0.0, -9.5)","(0, 0)"
1,0.250000,0.00,-9.50,none,"(0.0, -9.5)","(0, 0)"
2,0.300000,0.00,-9.50,none,"(0.0, -9.5)","(0, 0)"
3,0.350000,0.00,-9.50,none,"(0.0, -9.5)","(0, 0)"
4,0.390000,0.00,-9.50,none,"(0.0, -9.5)","(0, 0)"
...,...,...,...,...,...,...
1516,76.000000,-5.84,8.50,right,"(-5.84, 8.5)","(1, 0)"
1517,76.050003,-5.53,8.50,right,"(-5.53, 8.5)","(1, 0)"
1518,76.099998,-5.08,8.50,right,"(-5.08, 8.5)","(1, 0)"
1519,76.139999,-4.80,8.48,right,"(-4.8, 8.48)","(1, 0)"


In [20]:
processed_df = utils.preprocess_game_data(gamestate_df)

processed_df

,game_id,time_elapsed,Pacman_X,Pacman_Y
0,382,0.190000,0.00,-9.50
1,382,0.230000,0.00,-9.50
2,382,0.290000,0.00,-9.50
3,382,0.330000,0.00,-9.50
4,382,0.390000,0.00,-9.50
...,...,...,...,...
715289,1045,19.240000,-5.83,8.50
715290,1045,19.290001,-5.32,8.50
715291,1045,19.340000,-4.98,8.50
715292,1045,19.389999,-4.45,8.49


In [21]:
tensordf, mask, game_idx = utils.create_game_trajectory_tensor(processed_df)

In [35]:
tensordf.shap

torch.Size([662, 6320, 3])